# TCP en pratique: HTTP

TCP est utilisé par une multitude de protocoles réseau que vous utilisez (sans le savoir) tous les jours!

- Pages et services web: HTTP, Websocket
- Emails: SMTP, POP, IMAP
- Transfères de fichiers entre machine: FTP, SAMBA, AirDrop

Le plus répandu est le protocole HTTP. Il permet de manière relativement simple d'accéder à des pages web mais aussi à des services.

**Essayez de faire un clic droit sur la page, puis "afficher le code source de la page". Vous verrez du code HTML qui a servi à générer tout ce document!**

## Service utilisant le protocole HTTP: météo en ligne

Avec l'accès universel à internet depuis n'importe quelle machine (de l'ordinateur au réfrigérateur...), de nombreux services ont vu le jour. Certaines entreprise mettent à disposition des données au profit d'autres applications.

La météo est un service assez courant, utilisé par votre téléphone pour afficher la météo, ou par Alexa pour vous réveiller en vous disant s'il va falloir se convrir ou non...

Le code suivant récupère la météo actuelle grace au service [Open Weather Map](http://openweathermap.org/current):

In [ ]:
import requests

# URL du service de météo
REQUETE_URL = 'http://api.openweathermap.org/data/2.5/weather'
# Clé à utiliser pour accéder au service (rien n'est jamais complètement gratuit!)
ISN_KEY = '29c169de6611a15b7fa6df28811d9f72'
# Ville qui nous interesse
VILLE = 'Roquebrune-Cap-Martin'

# LA requete magique qui va tout faire!
reponse = requests.get(REQUETE_URL, params={'APPID': ISN_KEY, 'q': VILLE, 'units': 'metric'}).json()

# A ce stade, reponse contient un tas d'infos qu'on peut utiliser pour ce qu'on veut!
print("A {}, il fait actuellement {} degrés, avec {} % d'humidité.".format(
    VILLE,
    reponse['main']['temp'],
    reponse['main']['humidity']
))
reponse

## Les différentes tâches effectuée pour ce résultat

Le format d'une requete HTTP est le suivant:

```
http://<nom d'hote ou adresse IP>:<port de connection>/<chemin vers page ou service>?<nom de parametre>=<valeur>&<nom de parametre>=<valeur>...
```
Exemples:
- http://google.fr/ (hote: google.fr, port: sous-entendu 80, chemin: /, pas de parametre)
- http://mycoffeeservice.com:5000/docoffee?milk=false (hote: mycoffeeservice.com, port: 5000, chemin: /docoffee, parametre: milk=false)

La requête précédente correspond à ceci: http://api.openweathermap.org/data/2.5/weather?APPID=29c169de6611a15b7fa6df28811d9f72&units=metric&q=Roquebrune-Cap-Martin

### La résolution de l'adresse IP

Avant même de pouvoir se connecter au serveur, il faut connaitre l'adresse IP de celui-ci. La seule information que nous ayons est le nom du serveur correspondant (api.openweathermap.org).

Pour résoudre une adresse IP, il faut interroger un service de DNS. Chaque ordinateur connait généralement un ou plusieurs serveur DNS autour de lui. La commande `ipconfig /all` vous permet de connaitre cette information.

Python propose une fonction permettant d'intérroger le serveur DNS: la commande `gethostbyname` dans le module `socket`

In [ ]:
from socket import *
print(gethostbyname('api.openweathermap.org'))

### La connection TCP

Pour TCP on parle de connection: Le client (vous!) et le serveur doivent d'abord se connecter l'un à l'autre avant de pouvoir parler. En python, la librairie `socket` permet de directement créer des `socket`, objet représentant une connection et permettant d'envoyer et recevoir des données.

Pour les cadres suivant, vous pouvez utiliser la fonction `ouvrir_connection` suivante qui va établir une connection. La connaissance du fonctionnement des sockets n'est pas requis à votre niveau.

In [ ]:
from socket import *

def ouvrir_connexion(adresse, port):
    """Fonction qui va créer une socket pour vous"""
    assert isinstance(adresse, str), "l'adresse doit etre une chaine de caracteres"
    assert isinstance(port, int), "le port doit etre un entier positif entre 1 et 65535"
    
    # Creation d'une socket IPv4 (AF_INET) et TCP (SOCK_STREAM).
    sock = socket(AF_INET, SOCK_STREAM)
    
    # Juste pour éviter d'attendre des heures en cas d'erreur
    sock.settimeout(3)
    
    # Connection au serveur
    sock.connect((adresse, port))
    
    # C'est tout bon!
    return sock

def fermer_connexion(sock):
    sock.close()

def envoyer(sock, requete):
    """Envoie une chaine de caracteres à travers la socket/connection TCP"""
    data = requete.encode()
    sock.send(data)
    
def recevoir(sock, taille=10000):
    """Recevoir des données, la taille en option"""
    data = sock.recv(taille)
    return data.decode()


### Demander au serveur la météo?

Revenons à l'URL de la requete de météo. Comme aucun port n'est défini, on part du principe que celui ci est le port 80 (port standard du protocole HTTP).

On ouvre la connection, puis... quoi? Comment demander la météo? Gentillement?

In [ ]:
METEO_IP = gethostbyname('api.openweathermap.org')
conn = ouvrir_connexion(METEO_IP, 80)
envoyer(conn, 'Puis-je avoir la météo à RCM SVP?')
print(recevoir(conn))

# Ne jamais oublier de fermer les connexions qu'on ouvre!
fermer_connexion(conn)

### Couche APPLICATION et protocole HTTP

Une fois la connection TCP établie, la couche suivante est la couche application. Chaque application a son propre langage, et sa propre syntaxe pour communiquer.

Une commande HTTP GET simple a le format suivant:
```
GET <URL> HTTP/1.0\r\n\r\n
```

La réponse du serveur va toujours contenir sur la première ligne un code et un status correspondant.
Voici quelques erreurs courantes:
- 404: Page non trouvée
- 400: Mauvaise requete
- 200: OK, la page existe ou la requete est valide

Mettons de-suite en pratique!

In [ ]:
URL = '/data/2.5/weather?APPID=29c169de6611a15b7fa6df28811d9f72&units=metric&q=Roquebrune-Cap-Martin,fr'

# Créer une requete GET avec l'URL précédente
REQUETE = 'GET {} HTTP/1.0\r\n\r\n'.format(URL)

# OBTENIR l'IP du service METEO
METEO_ADRESSE = gethostbyname('api.openweathermap.org')

conn = ouvrir_connexion(METEO_ADRESSE, 80)
envoyer(conn, REQUETE)
reponse = recevoir(conn)
print(reponse)
fermer_connexion(conn)

### Comprendre une réponse HTTP

Une réponse HTTP contient:
- un certain nombre d'informations utiles en première partie (les Headers, en-têtes)
- puis une ligne vide
- et enfin la réponse réelle

Une fois la réponse réelle extraite, il faut savoir la lire! Pour cela, il faut regarder l'en-tête "Content-Type" qui indique de quel genre de réponse il s'agit. Ici, c'est une réponse au format `json`, très facile à lire! Utilisez la commande `json.loads` qui retourne un dictionnaire.

In [ ]:
# Séparer la réponse entre headers et contenu au niveau de la ligne vide
headers, contenu = reponse.split('\r\n\r\n')

# Charger le contenu avec la command json.loads
import json
meteo = json.loads(contenu)

meteo

# Coder un client de chat!!!

Specialement pour la classe d'ISN, un petit service de discussion ('chat' en anglais) a été créé.

Le principe de fonctionnement est extrêmement simple: chaque client ouvre une connection TCP vers le serveur (ici, disponible à l'adresse `vps.jonathanmartin.fr`, sur le port `80` (Habituellement, le port 80 est reservé au protocol HTTP, mais j'ai des soucis de Firewall avec d'autres ports...).

Ensuite il faut suivre le protocole suivant:

- Tous message consiste en une ligne terminée par le caractère `\n`
- Il existe 4 type de messages:
  - Les commandes, qui commancent toutes par `!`:
    - `!help` affiche un message d'aide
    - `!quit` ferme proprement la connection avec le serveur
    - `!join:<pseudo>` vous permet de rejoindre la chatroom et commencer à envoyer/recevoir des messages.
      Vous ne pouvez pas envoyer de message avant d'avoir rejoint le chat avec un pseudonyme!
    - `!list` affiche la listes des pseudos de clients déjà connectés (vos camarades?)
  - Les messages venant du serveur commencent par `%`, et existe uniquement à titre d'information
  - Les messages qui commence par `@<pseudo>:` sera envoyé uniquement au pseudo en question
  - Tout autre message envoyé à tout le monde.
  
Pour accélerer un peu le TP, je vous ai préparé un client déjà tout prêt, qui vous permet de vous connecter au serveur et envoyer des messages ou commandes.

**Essayez de vous connecter au serveur, de vous enregistrer avec votre nom, puis envoyer des messages pour vous familiariser avec la syntaxe!**

In [ ]:
from chat import ChatClient

try:
    # Au cas ou vous essayer de recharger ce bloc,
    # Il faut s'assurer qu'un client précédement ouvert est bien fermé
    client.stop()
except:
    pass

client = ChatClient('vps.jonathanmartin.fr', 80)
client.start()

## Ecrire un BOT de chat

Les "BOTS" sont des programmes autonomes qui se connectent automatiquement à des services et utilise les protocoles de communication à des fins plus ou moins douteuse... Le principe du BOT est d'exploiter des faiblesses d'un service ou protocole pour en abuser.

Ecrivons un bot simple, qui va:
- se connecter au serveur
- joindre le chat avec un pseudo généré au hasard (ex: troll65434567, random34567, ...)
- envoyer des messages un peu au hasard (soyez créatifs!)
- puis se déconnecte

In [ ]:
# A VOUS DE VOUS DEBROUILLER MAINTENANT!!!
# AIDEZ-VOUS DU CODE DES PRECEDENTS EXERCICES

# Creer une socket TCP vers le port 80 du serveur vps.jonathanmartin.fr

# Generer un pseudo au hasard

# Envoyer une command !join avec votre pseudo

# Recevoir et afficher la réponse

# Ecrire quelques messages

# Quitter la chatroom et fermer la socket